# SIMPET runs analysis

In [2]:
import yaml
import wandb
import pandas as pd
from pyprojroot import here

## Download configs from WANDB

In [66]:
SAVE_CONFIGS_PATH = here().joinpath("wandb_simpet_experiments_configs")
SAVE_CONFIGS_PATH.mkdir(parents=True, exist_ok=True)

api = wandb.Api()
configs = {run.name: [file for file in run.files() if file.name == "config.yaml"][0] for run in api.runs(path="fisikillo/SimPET-Randfigs-Simulations")}

for name, cfg in configs.items():
    if name != "GE Discovery ST":
        name = "".join(name.split(" ")[0:2])
    else:
        name = "".join(name.split(" ")[0:3])
    path = SAVE_CONFIGS_PATH.joinpath(name)
    cfg.download(path, replace=True)

## Read configs

In [67]:
configs = {}
REF_SCANNER = "GEDiscoveryST"

for cfg in SAVE_CONFIGS_PATH.rglob("**/*.yaml"):
    with open(cfg, 'r') as cfg_file:
        configs.update({cfg.parent.name: yaml.safe_load(cfg_file)})

scanner_configs = {name: cfg["params"]["value"]["scanner"] for name, cfg in configs.items()}
REF_CONFIG = scanner_configs.pop(REF_SCANNER)

## Show which parameters are different from a ref config that works

In [5]:
params_different_from_ref_config = {}

for name, s_cfg in scanner_configs.items():
    params_different = {k: f"{REF_CONFIG[k]}, {v}" + f"{(' D' if REF_CONFIG[k] != v else ' E')}" for k, v in s_cfg.items()}
    params_different_from_ref_config[name] = params_different

### Ref config (GE Discovery ST)

In [12]:
ref_cfg_df = pd.DataFrame([REF_CONFIG]).set_index("scanner_name").transpose()
ref_cfg_df.index.names = ["Parameters"]
ref_cfg_df

scanner_name,GE Discovery ST
Parameters,
simset_material,10
average_doi,0.84
scanner_radius,44.31
num_rings,24
axial_fov,15.7
z_crystal_size,0.63
transaxial_crystal_size,0.63
crystal_thickness,3
energy_resolution,16


In [70]:
scanner_df = pd.DataFrame([{**scanner_cfg, **{"scanner_name": name}} for name, scanner_cfg in params_different_from_ref_config.items()])
new_idx = sorted([sname for sname in scanner_df["scanner_name"]], key=lambda name: int(name.split("RandomScanner")[-1]))
new_idx.reverse()
ordered_scanner_df = scanner_df.set_index("scanner_name").loc[new_idx, ].transpose()
ordered_scanner_df.index.names = ["Parameters"]

# Show only incrmental configurations
n_cols = ordered_scanner_df.shape[-1]
configs_of_interest = ordered_scanner_df[[f"RandomScanner{i}" for i in reversed(range(4, n_cols))]]
configs_of_interest



scanner_name,RandomScanner12,RandomScanner11,RandomScanner10,RandomScanner9,RandomScanner8,RandomScanner7,RandomScanner6,RandomScanner5,RandomScanner4
Parameters,,,,,,,,,
simset_material,"10, 18 D","10, 18 D","10, 18 D","10, 10 E","10, 10 E","10, 10 E","10, 10 E","10, 10 E","10, 10 E"
average_doi,"0.84, 1.16 D","0.84, 1.16 D","0.84, 1.16 D","0.84, 1.05 D","0.84, 1.05 D","0.84, 0.84 E","0.84, 0.84 E","0.84, 0.84 E","0.84, 0.84 E"
scanner_radius,"44.31, 38.25 D","44.31, 38.25 D","44.31, 38.25 D","44.31, 42.51 D","44.31, 42.51 D","44.31, 42.51 D","44.31, 42.51 D","44.31, 42.51 D","44.31, 42.51 D"
num_rings,"24, 48 D","24, 48 D","24, 48 D","24, 41 D","24, 41 D","24, 41 D","24, 24 E","24, 24 E","24, 24 E"
axial_fov,"15.7, 22.01 D","15.7, 22.01 D","15.7, 22.01 D","15.7, 17.43 D","15.7, 17.43 D","15.7, 17.43 D","15.7, 17.43 D","15.7, 17.43 D","15.7, 15.7 E"
z_crystal_size,"0.63, 0.45 D","0.63, 0.45 D","0.63, 0.45 D","0.63, 0.41 D","0.63, 0.41 D","0.63, 0.41 D","0.63, 0.41 D","0.63, 0.41 D","0.63, 0.63 E"
transaxial_crystal_size,"0.63, 0.4 D","0.63, 0.4 D","0.63, 0.4 D","0.63, 0.6 D","0.63, 0.6 D","0.63, 0.6 D","0.63, 0.6 D","0.63, 0.6 D","0.63, 0.6 D"
crystal_thickness,"3, 2.93 D","3, 2.93 D","3, 2.93 D","3, 2.02 D","3, 2.02 D","3, 2.02 D","3, 3 E","3, 3 E","3, 3 E"
energy_resolution,"16, 12.61 D","16, 12.61 D","16, 12.61 D","16, 13.69 D","16, 16 E","16, 16 E","16, 16 E","16, 16 E","16, 16 E"


In [74]:
for col in configs_of_interest.columns:
    scanner = ordered_scanner_df[col]
    print(
        f"{col}: different to {REF_SCANNER}", '\n', 
        scanner[scanner.str.endswith("D").fillna(False)], 
        '\n\n',
        f"{col}: equal to {REF_SCANNER}", '\n', 
        scanner[scanner.str.endswith("E").fillna(False)], 
        '\n\n'
    )

RandomScanner12: different to GEDiscoveryST 
 Parameters
simset_material                    10, 18 D
average_doi                    0.84, 1.16 D
scanner_radius               44.31, 38.25 D
num_rings                          24, 48 D
axial_fov                     15.7, 22.01 D
z_crystal_size                 0.63, 0.45 D
transaxial_crystal_size         0.63, 0.4 D
crystal_thickness                 3, 2.93 D
energy_resolution               16, 12.61 D
num_aa_bins                      210, 300 D
num_td_bins                      249, 166 D
min_energy_window             375, 461.72 D
max_energy_window             675, 572.05 D
coincidence_window             11.7, 4.94 D
numberOfSubsets                     7, 25 D
analytical_att_correction            0, 1 D
stir_recons_att_corr                 1, 0 D
x_dir_filter_FWHM                  1.5, 0 D
y_dir_filter_FWHM                  1.5, 0 D
z_dir_filter_FWHM                    3, 0 D
max_segment                        23, 31 D
Name: RandomScanner